## Ray

Talk about the capabilities of ray in general briefly, and how this can make RL extra powerful for bigger problems. Basically, how does RLlib leverage ray (rollout workers etc)

also mention how tune (previous slide deck) fits in

RLlib config params that use ray, e.g.

        # num_workers: 16
        # num_gpus: 1
        num_envs_per_worker:
            grid_search: [16, 32]

and also the 

    "create_env_on_driver"  : True,
    
that  we've been ignoring the whole time